In [1418]:
import numpy as np
from typing import List, Dict, Union, Tuple
from tenpy.networks.site import SpinSite
from tenpy.linalg import np_conserved as npc
from tenpy.linalg.np_conserved import Array
from tenpy.networks.mpo import MPO
from tenpy.models.model import MPOModel
from tenpy.models.lattice import Chain
from tenpy.networks.mps import MPS  
from tenpy.algorithms.network_contractor import ncon

In [1419]:
def _dirichlet_weight(rng: np.random.Generator, n: int, alpha: float = 1.0) -> np.ndarray:
    """
    Return w ∈ R^n, w ≥ 0, sum(w) = 1.
    Samples w via independent Gamma(alpha,1) and normalizing.
    - alpha = 1.0  → uniform over the simplex (flat with no bias).
    - alpha < 1.0  → spikier (few large entries, many near zero).
    - alpha > 1.0  → smoother (entries more even).
    Edge cases:
      n = 0 → empty vector, n = 1 → [1.0].
    """

    #Sanity checks to avoid useless computation (R^0 and R vectors)
    if n == 0:
        return np.empty((0,), dtype=np.float64)
    if n == 1:
        return np.array([1.0], dtype=np.float64)
    #Invalid parameter
    if alpha <= 0:
        raise ValueError("dirichlet_alpha must be > 0.")
    
    
    #Gamma sampled vector of R^n
    g = rng.gamma(shape=alpha, scale=1.0, size=n)
    
    #Sum of the vector 
    s = g.sum()
    
    # numerically, s > 0 almost surely for alpha>0; still guard:
    if s == 0.0:
        # extremely unlikely; fallback to a one-hot at a random index
        w = np.zeros(n, dtype=np.float64)
        w[rng.integers(n)] = 1.0
        return w
    return g / s


# --------- Validators (keep constraints honest) ----------

def _check_L_row_substochastic(L: np.ndarray, atol: float = 1e-12):
    # For each (x,i): sum_j L[x,i,j] ≤ 1
    row_sums = L.sum(axis=2)  # (d, Dl)
    if np.any(row_sums > 1 + atol):
        raise AssertionError("random_L: found a row with sum > 1.")
    if np.any(L < -atol) or np.any(L > 1 + atol):
        raise AssertionError("random_L: entries must lie in [0,1].")

def _check_R_col_substochastic(R: np.ndarray, atol: float = 1e-12):
    # For each (x,j): sum_i R[x,i,j] ≤ 1
    col_sums = R.sum(axis=1)  # (d, Dr)
    if np.any(col_sums > 1 + atol):
        raise AssertionError("random_R: found a column with sum > 1.")
    if np.any(R < -atol) or np.any(R > 1 + atol):
        raise AssertionError("random_R: entries must lie in [0,1].")

def _check_box_0_1(A: np.ndarray, atol: float = 1e-12):
    if np.any(A < -atol) or np.any(A > 1 + atol):
        raise AssertionError("random_A1: entries must lie in [0,1].")

# --------- Substochastic generators (switched constraints) ---------
def random_L(
    d: int, Dl: int, Dr: int, seed=None, *,
    dirichlet_alpha: float = 1.0, check: bool = True
) -> np.ndarray:
    """
    Substochastic L_k, shape (d, Dl, Dr).
    Constraint : for each physical index x and each ROW i,
        sum_j L[x, i, j] ≤ 1  (row-substochastic)
    Construction: for each (x,i), draw a total mass s ~ U[0,1) and a Dirichlet
    weight w over Dr entries; set the row L[x,i,:] = s * w.
    """
    rng = np.random.default_rng(seed)
    L = np.zeros((d, Dl, Dr), dtype=np.float64)
    
    #Avoid unnecessary computation if degenerate input
    if d == 0 or Dl == 0 or Dr == 0:
        return L

    #Populate the rows substocastically
    for x in range(d):
        for i in range(Dl):
            s = rng.random()  # total mass in [0,1)
            if s == 0.0:
                continue      # zero row
            w = _dirichlet_weight(rng, Dr, alpha=dirichlet_alpha)  # sums to 1
            L[x, i, :] = s * w                                    # row sum = s ≤ 1

    if check:
        _check_L_row_substochastic(L)
    return L

def random_R(
    d: int, Dl: int, Dr: int, seed=None, *,
    dirichlet_alpha: float = 1.0, check: bool = True
) -> np.ndarray:
    """
    Substochastic R_k, shape (d, Dl, Dr).
    Constraint : for each physical x and each COLUMN j,
        sum_i R[x, i, j] ≤ 1  (column-substochastic)
    Construction: for each (x,j), draw mass s ~ U[0,1) and a Dirichlet over Dl
    entries; set the column R[x, :, j] = s * w.
    """
    rng = np.random.default_rng(seed)
    R = np.zeros((d, Dl, Dr), dtype=np.float64)
    
    #Avoid unnecessary computation if degenerate input
    if d == 0 or Dl == 0 or Dr == 0:
        return R
    
    #Populate the columns substocastically
    for x in range(d):
        for j in range(Dr):
            s = rng.random()  # total mass in [0,1)
            if s == 0.0:
                continue      # zero column
            w = _dirichlet_weight(rng, Dl, alpha=dirichlet_alpha)  # sums to 1
            R[x, :, j] = s * w                                    # column sum = s ≤ 1

    if check:
        _check_R_col_substochastic(R)
    return R

def random_A1(d: int, D: int, seed=None, *, check: bool = True) -> np.ndarray:
    """
    First site core A1, shape (d, 1, D).
    Continuous in [0,1] (uniform). To have occasional exact 1.0 values,
    we can optionally snap a tiny fraction to 1.0 after sampling.
    """
    rng = np.random.default_rng(seed)
    A = rng.uniform(0.0, 1.0, size=(d, 1, D)).astype(np.float64)  # in [0,1)
    
    # Optional: snap some values to 1.0 with tiny prob (disabled by default)
    # snap_mask = rng.random(A.shape) < 1e-4
    # A[snap_mask] = 1.0
    
    if check:
        _check_box_0_1(A)
    return A

def wrap_site_tensor(T: np.ndarray):
    """(d, Dl, Dr) -> npc.Array labels ['vL','p','vR'] shape (Dl, d, Dr)"""
    Dl = T.shape[1]
    Ai = np.transpose(T, (1, 0, 2))  # (Dl, d, Dr)
    return npc.Array.from_ndarray_trivial(Ai, labels=['vL', 'p', 'vR'])

def tenpy_sites_and_svs(d: int, right_dims):
    N = len(right_dims)
    S = (d - 1) / 2.0
    site = SpinSite(S=S, conserve=None)
    lattice = Chain(L=N, site=site)
    sites = lattice.mps_sites()
    svs = [np.ones(1, dtype=float)]
    svs += [np.ones(right_dims[i], dtype=float) for i in range(N - 1)]
    svs += [np.ones(1, dtype=float)]
    return sites, svs

In [1420]:
def site_tensor_from_Bi(Bi: dict, d: int, *, strict_keys: bool = True):
    """
    Costruisce T[x] = Bi[x] con fallback a zeri. T shape: (d, Dl, Dr).

    Se strict_keys=True, controlla che tutte le chiavi di Bi siano in [0, d-1].
    Richiede che Bi non sia vuoto.
    """
    if not Bi:
        raise ValueError("Bi non può essere vuoto: serve almeno una matrice per fissare (Dl, Dr).")

    if strict_keys:
        bad_keys = [k for k in Bi.keys() if not (0 <= k < d)]
        if bad_keys:
            raise ValueError(f"Chiavi fuori range [0, {d-1}]: {bad_keys}")

    # tutte le matrici devono avere la stessa shape 2D
    shapes = {np.shape(M) for M in Bi.values()}
    if len(shapes) != 1:
        raise ValueError(f"shape incoerenti in Bi: {shapes}")
    Dl, Dr = next(iter(shapes))
    if len((Dl, Dr)) != 2:
        raise ValueError(f"Le matrici in Bi devono essere 2D, shape trovata: {Dl, Dr}")

    # dtype comune (considera anche i fallback a 0)
    dtype = np.result_type(*[np.asarray(Bi.get(x, 0)).dtype for x in range(d)])

    T = np.zeros((d, Dl, Dr), dtype=dtype)
    zero_block = np.zeros((Dl, Dr), dtype=dtype)

    for x in range(d):
        Mx = np.asarray(Bi.get(x, zero_block), dtype=dtype)
        if Mx.shape != (Dl, Dr):
            raise ValueError(f"Bi[{x}] ha shape {Mx.shape}, atteso {(Dl, Dr)}")
        T[x] = Mx

    return T, Dl, Dr

def build_mps(B_list, d: int):
    # Controllo d intero positivo
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    d = int(d)

    N = len(B_list)
    if N == 0:
        raise ValueError("B_list non può essere vuoto.")

    tensors = []
    right_dims = []

    prev_Dr = None

    for i, Bi in enumerate(B_list):
        T, Dl, Dr = site_tensor_from_Bi(Bi, d)

        # Bordo sinistro
        if i == 0:
            if Dl != 1:
                raise ValueError(f"sito {i}: Dl deve essere 1 (bordo sinistro), trovato {Dl}")
        else:
            # Bond interno: Dl(i) deve coincidere con Dr(i-1)
            if Dl != prev_Dr:
                raise ValueError(
                    f"mismatch bond interno tra sito {i-1} e {i}: "
                    f"Dr[{i-1}]={prev_Dr}, Dl[{i}]={Dl}"
                )

        # Bordo destro
        if i == N - 1 and Dr != 1:
            raise ValueError(f"sito {i}: Dr deve essere 1 (bordo destro), trovato {Dr}")

        tensors.append(T)
        right_dims.append(Dr)
        prev_Dr = Dr

    A = [wrap_site_tensor(T) for T in tensors]
    sites, svs = tenpy_sites_and_svs(d, right_dims)

    return MPS(sites, A, svs, bc='finite', form='A')

In [1421]:
def build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu):
    pmid = 1 - pd - pu
    if not (0 <= pd <= 1 and 0 <= pu <= 1 and pd + pu <= 1):
        raise ValueError("Probabilità non valide: servono pd, pu >=0 e pd+pu <= 1")

    B_list = []

    # Sito 1: (1,2)
    B1 = {
        0: np.array([[ (S0/(N+1))*d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ]]),
        1: np.array([[ (S0/(N+1))*m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ]]),
        2: np.array([[ (S0/(N+1))*u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ]]),
    }
    B_list.append(B1)

    # Siti 2..N-1: (2,2)  ← QUI MANCANO NEL TUO FILE
    for i in range(2, N):
        Bi = {
            0: np.array([[ d_op*pd,  (S0/(N+1))*d_op*pd - K*pd ],
                         [     pd,                               1 ]]),
            1: np.array([[ m_op*pmid, ((S0/(N+1))*m_op - K)*pmid ],
                         [      pmid,                               1 ]]),
            2: np.array([[ u_op*pu,  (S0/(N+1))*u_op*pu - K*pu ],
                         [    pu,                                 1 ]]),
        }
        B_list.append(Bi)

    # Sito N: (2,1)
    BN = {
        0: np.array([[ d_op*pd ],
                     [     pd ]],),
        1: np.array([[ m_op*pmid ],
                     [      pmid ]]),
        2: np.array([[ u_op*pu ],
                     [     pu ]]),
    }
    B_list.append(BN)
    return B_list

In [1422]:
def build_mps_AR(d: int, N: int, D: int, seed=None):
    """
    Crea b(x) con bond dimensione D:
      A1: (d, 1, D)
      R2..R_{N-1}: (d, D, D)
      RN: (d, D, 1)
    """
    # controlli basilari
    if N < 2:
        raise ValueError("N deve essere >= 2 per avere A1 e almeno un R.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d)
    D = int(D)

    A1 = random_A1(d, d, seed=seed)

    Rs = []
    for k in range(2, N):
        sk = None if seed is None else seed + k
        Rs.append(random_R(d, Dl=D, Dr=D, seed=sk))

    sN = None if seed is None else seed + N
    RN = random_R(d, Dl=D, Dr=1, seed=sN)

    tensors = [A1] + Rs + [RN]

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]
    right_dims = [T.shape[2] for T in tensors]  # Dr per ciascun sito
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')

def build_mps_AR_bond(d: int, N: int, D: int, seed=None):
    """
    Crea un MPS psi con N siti (N dispari) e bond che crescono verso il centro
    come Dl -> min(Dl * d, D) e poi decrescono in modo simmetrico.

    Tensore sito i (0-based) ha shape (d, Dl_i, Dr_i).
    """
    # controlli basilari
    if N < 3 or N % 2 == 0:
        raise ValueError("N deve essere dispari e >= 3.")
    if d < 1 or int(d) != d:
        raise ValueError("`d` deve essere intero positivo.")
    if D < 1 or int(D) != D:
        raise ValueError("`D` deve essere intero positivo.")
    d = int(d)
    D = int(D)

    # half = indice del bond centrale
    half = N // 2  # per N=7 -> 3

    # costruiamo i bond da sinistra fino al centro
    bonds = [1]   # bond[0] = 1
    Dl = 1
    for k in range(1, half + 1):
        candidate = Dl * d
        Dr = candidate if candidate < D else D
        bonds.append(Dr)
        Dl = Dr

    # riflettiamo per avere simmetria: [1, ..., centro, ..., 1]
    # es: [1,3,9,10] -> [1,3,9,10,10,9,3,1]
    bonds = bonds + bonds[::-1]
    assert len(bonds) == N + 1  # N siti -> N+1 bond

    tensors = []

    # primo sito: Dl = bonds[0]=1, Dr = bonds[1]
    Dr0 = bonds[1]
    A1 = random_A1(d, Dr0, seed=seed)   # (d,1,Dr0)
    tensors.append(A1)

    # siti 1..N-1
    for i in range(1, N):
        Dl_i = bonds[i]
        Dr_i = bonds[i + 1]
        sk = None if seed is None else seed + i
        tensors.append(random_R(d, Dl=Dl_i, Dr=Dr_i, seed=sk))  # (d, Dl_i, Dr_i)

    # wrap TenPy
    A = [wrap_site_tensor(T) for T in tensors]  # (d,Dl,Dr) -> (Dl,d,Dr)
    right_dims = [T.shape[2] for T in tensors]  # Dr per sito
    sites, svs = tenpy_sites_and_svs(d, right_dims)
    return MPS(sites, A, svs, bc='finite', form='A')




In [1423]:
S0, K, N = 1.0, 0.2, 9
d_op, m_op, u_op = 0.9, 1.0, 3
pd, pu = 0.2, 0.4   

B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)

b = build_mps(B_list, d=3)

psi = build_mps_AR_bond(d=3, N=len(B_list), D=9)

val = b.overlap(psi)
print("sum_x psi(x)b(x) =", val)

sum_x psi(x)b(x) = 0.0006771138945436308


In [1424]:
def contract_left(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i-1)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == 1:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")


    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # A0: (-1, -2,  1)
    # A1: ( 1, -3,  2)
    # A2: ( 2, -4,  3)
    # ...
    # A_{i-2}: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d

    return T

def contract_right(tensor, i, N):
    A = [tensor.get_B(k) for k in range(i,N)]   # each: (Dl, d, Dr)
    #print(len(A))

    if i == N:
        return None   # oppure semplicemente "return" per terminare senza valore

    if i > N:
        raise ValueError(f"Indice i={i} maggiore del numero di siti N={N}.")
    
    if i <= 0:
        raise ValueError(f"Indice i={i} minore o uguale a 0.")

    # ncon index bookkeeping:
    # open legs use negative integers; positive integers are contracted
    # Ai: (-1, -2,  1)
    # Ai+1: ( 1, -3,  2)
    # Ai+2: ( 2, -4,  3)
    # ...
    # AN-1: (i-2, -(i), -(i+1))  # the final right bond stays open

    con_tensors = []
    con_indices = []
    if len(A) == 1:
        con_indices.append([-1, -2, -3])
        con_tensors.append(A[0])
    else:
        for k, Ak in enumerate(A):
            if k == 0:                                  # first site
                con_tensors.append(Ak)
                con_indices.append([-1, -2, 1])
            elif k < len(A) - 1:                        # middle sites
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), k+1])
            else:                                       # last of the block
                con_tensors.append(Ak)
                con_indices.append([k, -(k+2), -(k+3)])
    #print(con_indices)

    # result has open legs: [-1 (left=1), -2..-(i) (physicals), -(i+1) (right bond Di-1)]
    T = ncon(con_tensors, con_indices)
    # T.shape == (1, d, d, ..., d, D_{i-1})   # (i-1) copies of d
    return T

UL = contract_left(tensor=psi,i=3,N=len(B_list))
UR = contract_right(tensor=psi,i=3,N=len(B_list))


In [1425]:
def tensorial_derivative(psi, b, site):

    N_psi = psi.L
    N_b  = b.L
    if N_psi != N_b:
        raise ValueError(f"psi e b hanno lunghezze diverse: {N_psi} vs {N_b}")
    N = N_psi

    if not (1 <= site <= N):
        raise ValueError(f"site={site} fuori range [1, {N}]")
    
    UL = contract_left(tensor=psi,i=site,N=N)
    UR = contract_right(tensor=psi,i=site,N=N)
    BL = contract_left(tensor=b,i=site,N=N)
    BR = contract_right(tensor=b,i=site,N=N)

    if UL is not None and len(UL.shape) != len(BL.shape):
        raise ValueError(f"Dimensione di UL diversa da BL")
    if UR is not None and len(UR.shape) != len(BR.shape):
        raise ValueError(f"Dimensione di UR diversa da BR")

    if UL is not None:
        left_tensors = [UL,BL]
        left_links = [
            [-1] + [k for k in range(1,site)] + [-2], 
        [-3] + [k for k in range(1,site)] + [-4] 
        ]
        #print('left contraction links',left_links)
        left_contraction = ncon(left_tensors,left_links) 
    ## Il risultato è un tensore di dimensione (1,D_i;1,2)

    if UR is not None:
        right_tensors = [UR,BR]
        right_links = [
            [-1] + [k for k in range(1,N-site+1)] + [-2], 
        [-3] + [k for k in range(1,N-site+1)] + [-4] 
        ]
        #print('right contraction links',right_links)
        right_contraction = ncon(right_tensors,right_links) 
    ## Il risultato è un tensore di dimensione (D_i+1,1;2,1)

    if site > 1 and site < N:
        final_tensors = [left_contraction, b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [2],
            [-5] + [-6] + [2] + [-7]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == 1:
        final_tensors = [b.get_B(site-1),right_contraction]
        final_links = [
            [-1] + [-2] + [1],
            [-3] + [-4] + [1] + [-5]
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    elif site == N:
        final_tensors = [left_contraction, b.get_B(site-1)]
        final_links = [
            [-1] + [-2] + [-3] + [1],
            [1] + [-4] + [-5],
        ]
        final_contraction = ncon(final_tensors,final_links)
        #print(final_links)
        #print(final_contraction.shape)

    return np.squeeze(final_contraction)  

In [1426]:
derivata = tensorial_derivative(psi=psi,b=b, site=9)
print(derivata.shape)

(3, 3)


In [1427]:
def newupdate(
    G: np.ndarray,
    *,
    zero_if_nonpositive: bool = False,
    dtype=np.uint8,
) -> np.ndarray:
    """
    Given a 2D array G (shape m x n), return X (m x n) where each column j is
    one-hot at the index of the maximum entry of G[:, j]. If zero_if_nonpositive=True
    and the column's maximum <= 0, the whole column is set to zeros.

    - NaNs in G are treated as -inf (ignored for maxima).
    - Ties are broken by np.argmax's rule (first occurrence).

    Parameters
    ----------
    G : np.ndarray
        Gradient / environment matrix of shape (m, n).
    zero_if_nonpositive : bool, default True
        If True, columns whose maximum <= 0 become all zeros.
        If False, columns are always one-hot at the argmax (even if max <= 0).
    dtype : numpy dtype, default np.uint8
        Dtype of the output (0/1).

    Returns
    -------
    X : np.ndarray
        One-hot selection matrix of shape (m, n).
    """
    if G.ndim != 2:
        raise ValueError("G must be a 2D array (m x n).")

    # Treat NaNs as -inf so they never win the argmax
    G_clean = np.where(np.isnan(G), -np.inf, G)

    # Argmax per column (shape: (n,))
    idx = np.argmax(G_clean, axis=0)

    # Build one-hot result
    m, n = G_clean.shape
    X = np.zeros((m, n), dtype=dtype)
    if n > 0:
        X[idx, np.arange(n)] = 1

    if zero_if_nonpositive:
        col_max = np.max(G_clean, axis=0)  # (n,)
        mask = col_max > 0                 # keep only columns with strictly positive max
        if not np.all(mask):
            # zero out columns that don't pass the positivity test
            X[:, ~mask] = 0

    return X

In [1428]:
def SweepingAlgorithm(b, d, D, err):
    '''
    Input:
        b: lista di tensori
        d: dimensione fisica della Tensor Network
        D: bond dimension della tensor Network di psi
    '''

    # Cominciamo con l'ansatz del tipo A R R ... R
    N = b.L
    psi = build_mps_AR_bond(d=d, N=N, D=D)
    dims = [psi.get_B(i).shape for i in range(psi.L)]
    print(dims)
    k0 = b.overlap(psi)

    site = 1
    A_tilde = tensorial_derivative(psi=psi, b=b, site=site)
    A_prime = A_tilde.to_ndarray()
    L_out = newupdate(A_prime)
    d_phys, Dr = L_out.shape
    L = L_out.reshape(1, d_phys, Dr)
    L_tenpy = npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR'])
    psi.set_B(site-1, L_tenpy)
    k1 = b.overlap(psi)
    nstep = 1

    while True:

        for i in range(N - 1):
            site = i + 1

            if site == 1 and nstep == 1:
                continue

            A_tilde = tensorial_derivative(psi=psi, b=b, site=site)
            A_prime = A_tilde.to_ndarray()

            if site > 1 and site < N:
                Dl, d_phys, Dr = A_prime.shape
                A_prime = A_prime.reshape(Dl * d_phys, Dr)
                L_out = newupdate(A_prime)
                L = L_out.reshape(Dl, d_phys, Dr)
            else:
                d_phys, Dr = A_prime.shape
                L_out = newupdate(A_prime)
                L = L_out.reshape(1, d_phys, Dr)

            L_tenpy = npc.Array.from_ndarray_trivial(L, labels=['vL', 'p', 'vR'])
            psi.set_B(i, L_tenpy)

            k0 = k1
            k1 = b.overlap(psi)
            nstep += 1 
            #print(nstep)
            if np.abs(k1 - k0) <= err * max(np.abs(k0), 1e-12):
                return psi, k1, nstep 

        for j in range(N - 1, 0, -1):
            site = j + 1
            A_tilde = tensorial_derivative(psi=psi, b=b, site=site)
            A_prime = A_tilde.to_ndarray()

            if site > 1 and site < N:
                Dl, d_phys, Dr = A_prime.shape
                A_prime = A_prime.reshape(Dl, Dr*d_phys)
                R_out = newupdate(A_prime.T).T
                #print(nstep,A_prime.shape,R_out.shape)
                R = R_out.reshape(Dl, d_phys, Dr)
            else:
                Dl, d_phys = A_prime.shape
                R_out = newupdate(A_prime.T).T                    
                R = R_out.reshape(Dl, d_phys, 1)

            R_tenpy = npc.Array.from_ndarray_trivial(R, labels=['vL', 'p', 'vR'])
            psi.set_B(j, R_tenpy)

            k0 = k1
            k1 = b.overlap(psi)
            print(k1)
            nstep += 1
            #print(nstep)
            if np.abs(k1 - k0) <= err * max(np.abs(k0), 1e-12):
                return psi, k1, nstep

In [1429]:
N = 15
B_list = build_B_list(S0, K, N, d_op, m_op, u_op, pd, pu)
b = build_mps(B_list, d=3)
psi , K, nstep = SweepingAlgorithm(b=b, d=3, D=10, err=1e-18)
print(nstep)

[(1, 3, 3), (3, 3, 9), (9, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 10), (10, 3, 9), (9, 3, 3), (3, 3, 1)]
-0.004322842667239356
-0.012968528001718068
-0.014409475557464517
-0.014277115243717112
-0.013905854657584184
-0.01300449235920374
-0.0125702471889758
-0.011642658792797057
-0.01112676455340847
-0.009922197541795599
-0.009637680039645902
-0.009637680039645904
-0.008673912035681311
-0.0028913040118937706
29
